In [1]:
import polars as pl
import os
import argparse
import gzip

In [3]:
merge_columns = ["aminoAcid", "vFamilyName"]
count_column = "count (templates/reads)"
input_files= ["results/cleanup/F5302E/F5302E_productive.tsv.gz", "results/cleanup/F5302N/F5302N_productive.tsv.gz"]

In [12]:
aggregated_data = None
total_lines_ingested = 0
numeric_columns = []



In [14]:
dfs = []

for file_index, file_path in enumerate(input_files):
    sample_name = f"count_sample{file_index + 1}"
    numeric_columns.append(sample_name)

    with gzip.open(file_path, "rt") if file_path.endswith(".gz") else open(file_path, "r") as f:
        df = pl.read_csv(
            f,
            separator="\t",
            columns=merge_columns + [count_column]
        )

    df = df.rename({count_column: sample_name})
    dfs.append(df)

# Stack all files vertically
combined = pl.concat(dfs, how="diagonal")

# SQL-style GROUP BY
aggregated_data = (
    combined
    .group_by(merge_columns)
    .agg(pl.col(pl.NUMERIC_DTYPES).sum())
    .sort(merge_columns)
)


aggregated_data

/var/folders/bj/8722bm4j7tj2r8twwzwxs5z00000gn/T/ipykernel_14464/1108308465.py:24: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  .agg(pl.col(pl.NUMERIC_DTYPES).sum())


aminoAcid,vFamilyName,count_sample1,count_sample2
str,str,i64,i64
"""ADRHNQPQHF""","""TCRBV28""",0,1
"""CAAADPAVTNSPLHF""","""TCRBV10""",0,1
"""CAAAIGGAQDTQYF""","""TCRBV05""",1,0
"""CAAARDRATGELFF""","""TCRBV10""",0,1
"""CAAASAGEGYEQYF""","""TCRBV10""",1,0
…,…,…,…
"""YKTGRAENTEAFF""","""TCRBV17""",1,0
"""YSSDARLARGLTYEQYF""","""TCRBV17""",1,0
"""YSSGEESSYEQYF""","""TCRBV17""",0,1
